In [43]:
# import library
import pandas as pd
import numpy as np 
import matlab 
import matplotlib as plt
import os

'2.0.0'

In [24]:
# read the file
# mortality
mortality_path = r'data/Morticd10_part1.csv'
mortality_data_2002 = pd.read_csv(mortality_path, low_memory=False)
mortality_data_2002.columns.values

array(['Country', 'Admin1', 'SubDiv', 'Year', 'List', 'Cause', 'Sex',
       'Frmat', 'IM_Frmat', 'Deaths1', 'Deaths2', 'Deaths3', 'Deaths4',
       'Deaths5', 'Deaths6', 'Deaths7', 'Deaths8', 'Deaths9', 'Deaths10',
       'Deaths11', 'Deaths12', 'Deaths13', 'Deaths14', 'Deaths15',
       'Deaths16', 'Deaths17', 'Deaths18', 'Deaths19', 'Deaths20',
       'Deaths21', 'Deaths22', 'Deaths23', 'Deaths24', 'Deaths25',
       'Deaths26', 'IM_Deaths1', 'IM_Deaths2', 'IM_Deaths3', 'IM_Deaths4'],
      dtype=object)

In [25]:
# population
pop_path = r'data/pop.csv'
pop_data = pd.read_csv(pop_path, low_memory=False)
pop_data.columns.values

array(['Country', 'Admin1', 'SubDiv', 'Year', 'Sex', 'Frmat', 'Pop1',
       'Pop2', 'Pop3', 'Pop4', 'Pop5', 'Pop6', 'Pop7', 'Pop8', 'Pop9',
       'Pop10', 'Pop11', 'Pop12', 'Pop13', 'Pop14', 'Pop15', 'Pop16',
       'Pop17', 'Pop18', 'Pop19', 'Pop20', 'Pop21', 'Pop22', 'Pop23',
       'Pop24', 'Pop25', 'Pop26', 'Lb'], dtype=object)

In [4]:
# country code
country_code_path = r'data/country_codes.csv'
country_code = pd.read_csv(country_code_path, low_memory=False)
country_code

,country,name
0,1010,Algeria
1,1020,Angola
2,1025,Benin
3,1030,Botswana
4,1035,Burkina Faso
...,...,...
222,5197,Samoa
223,5198,Solomon Islands
224,5200,Tonga
225,5205,Tuvalu


In [26]:
# changes to the mortality data file, creating a new data frame with only the number of deaths and the population, country, year, deaths1 - deaths26, pop1 - pop 26, im_death
# creating the country code map 
country_code_map = {}
for index, rows in country_code.iterrows():
    country_code_map[rows['country']] = rows['name']

In [30]:
# the modified dataframe to work with 
modified_df = pd.DataFrame(columns = ['Country', 'Year', 'Deaths1', 'Deaths2', 'Deaths3', 'Deaths4',
       'Deaths5', 'Deaths6', 'Deaths7', 'Deaths8', 'Deaths9', 'Deaths10',
       'Deaths11', 'Deaths12', 'Deaths13', 'Deaths14', 'Deaths15',
       'Deaths16', 'Deaths17', 'Deaths18', 'Deaths19', 'Deaths20',
       'Deaths21', 'Deaths22', 'Deaths23', 'Deaths24', 'Deaths25',
       'Deaths26', 'Pop1',
       'Pop2', 'Pop3', 'Pop4', 'Pop5', 'Pop6', 'Pop7', 'Pop8', 'Pop9',
       'Pop10', 'Pop11', 'Pop12', 'Pop13', 'Pop14', 'Pop15', 'Pop16',
       'Pop17', 'Pop18', 'Pop19', 'Pop20', 'Pop21', 'Pop22', 'Pop23',
       'Pop24', 'Pop25', 'Pop26'])

In [42]:
# looping through the code map to locate each different countries
# condense the mortality and pop rate 
for key in country_code_map:
    temp_mortality_df = mortality_data_2002[mortality_data_2002['Country'] == key]
    if temp_mortality_df.empty:
        continue
    else:
        for i in range (1980, 2003):
            temp_temp_mortality_df = temp_mortality_df.loc[temp_mortality_df['Year'] == i]
            if temp_temp_mortality_df.empty:
                continue
            else:
                
                modified_df = modified_df.concat(pd.Series(), ignore_index=True)
                modified_df.loc[len(modified_df) - 1]['Country'] = key
                modified_df.loc[len(modified_df) - 1]['Year'] = i
    
                for index, row in temp_temp_mortality_df:
                    column_name = modified_df.columns.values 
                    for c in column_name[2:]:
                        if c in temp_temp_mortality_df.columns.values:
                            modified_df.loc[len(modified_df) - 1][c] = temp_temp_mortality_df[c].sum()
            
                # modified_df.index = modified_df.index + 1
                # modified_df = modified_df.sort_index()
                    
modified_df        
                    
        

       

AttributeError: 'DataFrame' object has no attribute 'append'